# 20.7 Challenge - MLB Home Team Runs Scored

Now that you've learned about random forests and decision trees let's do an exercise in accuracy. You know that random forests are basically a collection of decision trees. But how do the accuracies of the two models compare?

So here's what you should do. Pick a dataset. It could be one you've worked with before or it could be a new one. Then build the best decision tree you can.

Now try to match that with the simplest random forest you can. For our purposes measure simplicity with runtime. Compare that to the runtime of the decision tree. This is imperfect but just go with it.

Hopefully out of this you'll see the power of random forests, but also their potential costs. Remember, in the real world you won't necessarily be dealing with thousands of rows. It could be millions, billions, or even more.

In [215]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [231]:
mlb2016 = pd.read_csv(r'C:\Users\kylem\Desktop\Thinkful Data Science Projects\baseball_reference_2016_clean.csv')

In [232]:
pd.set_option('display.max_columns', None)
mlb2016.head()

,Unnamed: 0,attendance,away_team,away_team_errors,away_team_hits,away_team_runs,date,field_type,game_type,home_team,home_team_errors,home_team_hits,home_team_runs,start_time,venue,day_of_week,temperature,wind_speed,wind_direction,sky,total_runs,game_hours_dec,season,home_team_win,home_team_loss,home_team_outcome
0,0,40030.0,New York Mets,1,7,3,2016-04-03,on grass,Night Game,Kansas City Royals,0,9,4,7:38 p.m. Local,Kauffman Stadium,Sunday,74.0,14.0,from Right to Left,Sunny,7,3.216667,regular season,1,0,Win
1,1,21621.0,Philadelphia Phillies,0,5,2,2016-04-06,on grass,Night Game,Cincinnati Reds,0,8,3,7:11 p.m. Local,Great American Ball Park,Wednesday,55.0,24.0,from Right to Left,Overcast,5,2.383333,regular season,1,0,Win
2,2,12622.0,Minnesota Twins,0,5,2,2016-04-06,on grass,Night Game,Baltimore Orioles,0,9,4,7:07 p.m. Local,Oriole Park at Camden Yards,Wednesday,48.0,7.0,out to Leftfield,Unknown,6,3.183333,regular season,1,0,Win
3,3,18531.0,Washington Nationals,0,8,3,2016-04-06,on grass,Night Game,Atlanta Braves,1,8,1,7:10 p.m. Local,Turner Field,Wednesday,65.0,10.0,from Right to Left,Cloudy,4,2.883333,regular season,0,1,Loss
4,4,18572.0,Colorado Rockies,1,8,4,2016-04-06,on grass,Day Game,Arizona Diamondbacks,0,8,3,12:40 p.m. Local,Chase Field,Wednesday,77.0,0.0,in unknown direction,In Dome,7,2.650000,regular season,0,1,Loss


In [233]:
mlb2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463 entries, 0 to 2462
Data columns (total 26 columns):
Unnamed: 0           2463 non-null int64
attendance           2460 non-null float64
away_team            2463 non-null object
away_team_errors     2463 non-null int64
away_team_hits       2463 non-null int64
away_team_runs       2463 non-null int64
date                 2463 non-null object
field_type           2463 non-null object
game_type            2463 non-null object
home_team            2463 non-null object
home_team_errors     2463 non-null int64
home_team_hits       2463 non-null int64
home_team_runs       2463 non-null int64
start_time           2463 non-null object
venue                2463 non-null object
day_of_week          2463 non-null object
temperature          2463 non-null float64
wind_speed           2463 non-null float64
wind_direction       2463 non-null object
sky                  2463 non-null object
total_runs           2463 non-null int64
game_hours_dec  

In [234]:
mlb2016.drop(['date', 'day_of_week', 'start_time', 'Unnamed: 0', 'home_team_errors', 'game_hours_dec', 'home_team_loss', 'home_team_outcome'], axis=1, inplace=True)

In [235]:
pd.get_dummies(mlb2016)

,attendance,away_team_errors,away_team_hits,away_team_runs,home_team_hits,home_team_runs,temperature,wind_speed,total_runs,home_team_win,away_team_Arizona Diamondbacks,away_team_Atlanta Braves,away_team_Baltimore Orioles,away_team_Boston Red Sox,away_team_Chicago Cubs,away_team_Chicago White Sox,away_team_Cincinnati Reds,away_team_Cleveland Indians,away_team_Colorado Rockies,away_team_Detroit Tigers,away_team_Houston Astros,away_team_Kansas City Royals,away_team_Los Angeles Angels of Anaheim,away_team_Los Angeles Dodgers,away_team_Miami Marlins,away_team_Milwaukee Brewers,away_team_Minnesota Twins,away_team_New York Mets,away_team_New York Yankees,away_team_Oakland Athletics,away_team_Philadelphia Phillies,away_team_Pittsburgh Pirates,away_team_San Diego Padres,away_team_San Francisco Giants,away_team_Seattle Mariners,away_team_St. Louis Cardinals,away_team_Tampa Bay Rays,away_team_Texas Rangers,away_team_Toronto Blue Jays,away_team_Washington Nationals,field_type_on grass,field_type_on turf,game_type_Day Game,game_type_Night Game,home_team_Arizona Diamondbacks,home_team_Atlanta Braves,home_team_Baltimore Orioles,home_team_Boston Red Sox,home_team_Chicago Cubs,home_team_Chicago White Sox,home_team_Cincinnati Reds,home_team_Cleveland Indians,home_team_Colorado Rockies,home_team_Detroit Tigers,home_team_Houston Astros,home_team_Kansas City Royals,home_team_Los Angeles Angels of Anaheim,home_team_Los Angeles Dodgers,home_team_Miami Marlins,home_team_Milwaukee Brewers,home_team_Minnesota Twins,home_team_New York Mets,home_team_New York Yankees,home_team_Oakland Athletics,home_team_Philadelphia Phillies,home_team_Pittsburgh Pirates,home_team_San Diego Padres,home_team_San Francisco Giants,home_team_Seattle Mariners,home_team_St. Louis Cardinals,home_team_Tampa Bay Rays,home_team_Texas Rangers,home_team_Toronto Blue Jays,home_team_Washington Nationals,venue_AT&T Park,venue_Angel Stadium of Anaheim,venue_Busch Stadium III,venue_Chase Field,venue_Citi Field,venue_Citizens Bank Park,venue_Comerica Park,venue_Coors Field,venue_Dodger Stadium,venue_Fenway Park,venue_Fort Bragg Park,venue_Globe Life Park in Arlington,venue_Great American Ball Park,venue_Kauffman Stadium,venue_Marlins Park,venue_Miller Park,venue_Minute Maid Park,venue_Nationals Park,venue_Oakland-Alameda County Coliseum,venue_Oriole Park at Camden Yards,venue_PNC Park,venue_Petco Park,venue_Progressive Field,venue_Rogers Centre,venue_Safeco Field,venue_Target Field,venue_Tropicana Field,venue_Turner Field,venue_U.S. Cellular Field,venue_Wrigley Field,venue_Yankee Stadium III,wind_direction_ from Left to Right,wind_direction_ from Right to Left,wind_direction_ in from Centerfield,wind_direction_ in from Leftfield,wind_direction_ in from Rightfield,wind_direction_ in unknown direction,wind_direction_ out to Centerfield,wind_direction_ out to Leftfield,wind_direction_ out to Rightfield,sky_Cloudy,sky_Drizzle,sky_In Dome,sky_Night,sky_Overcast,sky_Rain,sky_Sunny,sky_Unknown,season_post season,season_regular season
0,40030.0,1,7,3,9,4,74.0,14.0,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,21621.0,0,5,2,8,3,55.0,24.0,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,12622.0,0,5,2,9,4,48.0,7.0,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
3,18531.0,0,8,3,8,1,65.0,10.0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [236]:
#lets build a decision tree of home team wins from the dataset

from sklearn import metrics
from sklearn import tree
import timeit
from datetime import timedelta

X = mlb2016.drop('home_team_win', 1)
Y = mlb2016['home_team_win']
X = pd.get_dummies(X)
X = X.dropna(axis=1)


In [237]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.3, random_state=0)

In [238]:
# measure our model's runtime
import time
from sklearn.tree import DecisionTreeClassifier
start_time = time.time()

# Initialize our decision tree.
clf = DecisionTreeClassifier(
    criterion='entropy',
    max_features=1,
    max_depth=4,
)

# Train Decision Tree Classifer
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
clf_score = cross_val_score(clf, X, Y, cv=10)
display(clf_score)
print("Decision Tree Accuracy: %0.2f (+/- %0.2f)" % (clf_score.mean(), clf_score.std() * 2))

# display model runtime
print("Runtime: %s seconds" % round(time.time() - start_time, 2))

array([0.51821862, 0.5708502 , 0.5465587 , 0.52631579, 0.53658537,
       0.53658537, 0.53252033, 0.54878049, 0.53658537, 0.54285714])

Decision Tree Accuracy: 0.54 (+/- 0.03)
Runtime: 0.07 seconds


### Forest

In [239]:
# measure our model's runtime
import time
start_time = time.time()

# Initialize our decision tree.
rfc = ensemble.RandomForestClassifier(n_estimators=10)

# Train Random Forest Classifer
rfc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = rfc.predict(X_test)

# Model Accuracy, how often is the classifier correct?
rfc_score = cross_val_score(rfc, X, Y, cv=10)
display(rfc_score)
print("Random Forest Accuracy: %0.2f (+/- %0.2f)" % (rfc_score.mean(), rfc_score.std() * 2))

# display model runtime
print("Runtime: %s seconds" % round(time.time() - start_time, 2))

array([0.97165992, 0.96761134, 0.96356275, 0.96761134, 0.95934959,
       0.96341463, 0.95934959, 0.9796748 , 0.98373984, 0.96734694])

Random Forest Accuracy: 0.97 (+/- 0.02)
Runtime: 0.31 seconds


Whoa... not sure if this is for real or if I've made a mistake somewhere. The jump in accuracy is quite astounding, and I'm not sure if it is legitimate. However, for the sake of the assignment - Comparing our tree to our forest, we can see that comparatively, at least, the forest takes much longer to run and is much more accurate. 